### Search Engine With Agents and Tools

to find all langchain integrations you can visit link below

https://python.langchain.com/docs/integrations/providers/

In [11]:
%pip install pymupdf --quiet
%pip install arxiv --quiet
%pip install wikipedia --quiet
%pip install langchain_huggingface --quiet
%pip install langchain_community --quiet
%pip install langchain_text_splitters --quiet
%pip install pip install faiss-cpu --quiet
%pip install langchain_groq --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


Note: you may need to restart the kernel to use updated packages.


In [6]:
## Preparing the tools

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

doc_content_chars_max = 250
top_k_results = 1

tools = []

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=top_k_results,doc_content_chars_max=doc_content_chars_max,lang='en',wiki_client=None)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
tools.append(wiki)

print('wiki.name =',wiki.name)

api_wrapper_arxiv = ArxivAPIWrapper(arxiv_search=None,arxiv_exceptions=None,top_k_results=top_k_results,doc_content_chars_max=doc_content_chars_max)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
tools.append(arxiv)

print('arxiv.name =',arxiv.name)


wiki.name = wikipedia
arxiv.name = arxiv


In [ ]:
## Custom tool [RAG Document Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN','')
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLm-L6-v2')

In [8]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
%pip install -qU langchain-community faiss-cpu
vertordb = FAISS.from_documents(documents,embeddings)
retriever = vertordb.as_retriever()
print('retriever.name =',retriever)

Note: you may need to restart the kernel to use updated packages.
retriever.name = tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B0CA577D40> search_kwargs={}


In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any Information")
tools.append(retriever_tool)
print('retriever_tool.name =',retriever_tool.name)

retriever_tool.name = langsmith-search


In [14]:
## llm
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY","")
grok_api_key = os.getenv("GROQ_API_KEY")
model_name = "Llama3-8b-8192"
loaderDir = "research_papers"

llm = ChatGroq(model=model_name)

In [16]:
from langchain import hub
from langchain.agents import create_openai_tools_agent,AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")
print('prompt.messages =',prompt.messages)

agent = create_openai_tools_agent(llm,tools,prompt)
print('agent =',agent)

agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
print('agent_executor =',agent_executor)

prompt.messages = [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]
agent = first=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
}) middle=[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Ta

In [20]:
## useage

agentRes = agent_executor.invoke(
    {
        "input": "What is Large Language Model?",
    }
)

print('agentRes =',agentRes)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Large Language Model'}`


Page: Large language model
Summary: A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-
Invoking: `wikipedia` with `{'query': 'Large Language Model#Applications'}`


Page: Large language model
Summary: A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-A large language model is a type of artificial intelligence (AI) designed to process and generate human-like language. It is trained on vast amounts of data, allowing it to learn patterns and relationships in language, and to generate text that is coherent and natural-sounding. Large language models can be used fo